# Parallelization

Even for large cases _foxes_ calculations are fast, thanks to

- **Vectorization:** The states (and also the points, in the case of data calculation at evaluation points) are split into so-called _chunks_, which are sub-arrays of the large original data.
- **Parallelization:** These chunks are being sent to individual processes for calculation. Those calculations can be carried out simultaneously, i.e., _in parallel_.  

Vectorization and parallelization are managed by so-called _engines_ in _foxes_. If you do not explicitly specify the engine, a default will be chosen. This means that even if you do not know or care about _foxes_ engines, your calculations will be vectorized and parallelized.

## Available engines

These are the currently available engines, where each can be addressed by the short name or the full class name:

| Short name    | Class name         | Base package | Description                        |
|---------------|--------------------|--------------|------------------------------------| 
| threads       | ThreadsEngine      | [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) | Runs on a workstation/laptop,  sends chunks to threads |
| process       | ProcessEngine      | [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) | Runs on a workstation/laptop,  sends chunks to parallel processes |
| multiprocess  | MultiprocessEngine | [multiprocess](https://github.com/uqfoundation/multiprocess) |  Runs on a workstation/laptop,  sends chunks to parallel processes |
| ray  | RayEngine | [ray](hhttps://docs.ray.io/en/latest/) |  Runs on a workstation/laptop,  sends chunks to parallel processes |
| dask          | DaskEngine         | [dask](https://www.dask.org/) | Runs on a workstation/laptop, using processes or threads |
| local_cluster | LocalClusterEngine | [distributed](https://distributed.dask.org/en/stable/) | Runs on a workstation/laptop, creates a virtual local cluster |
| slurm_cluster | SlurmClusterEngine | [dask_jobqueue](https://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html) | Runs on a multi-node HPC cluster which is using SLURM |
| mpi           | MPIEngine          | [mpi4py](https://mpi4py.readthedocs.io/en/stable/index.html) | Runs on laptop/workstation/cluster, also supports multi-node runs |
| numpy         | NumpyEngine        | [numpy](https://numpy.org/) | Runs a loop over chunks, without parallelization |
| single        | SingleChunkEngine  | [numpy](https://numpy.org/) | Runs all in a single chunk, without parallelization |
| default       | DefaultEngine      | [numpy](https://numpy.org/), [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) | Runs either the `single` or the `process` engine, depending on the case size |

Note that the external packages are not installed by default. You can install them manually on demand, or use the option `pip install foxes[eng]` for the complete installation of all requirements of the complete list of engines.

Furthermore, scripts that use the `mpi` engine have to be started in a special way. For example, when running a script named `run.py` on 12 processors, the terminal command is
```console
mpiexec -n 12 python -m mpi4py.futures run.py
```

## Default engine

Let's start by importing _foxes_ and other required packages:

In [ ]:
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV

Next, we create a random wind farm and a random time series:

In [ ]:
n_times = 5000
n_turbines = 100
seed = 42

sdata = foxes.input.states.create.random_timseries_data(
    n_times,
    seed=seed,
)
states = foxes.input.states.Timeseries(
    data_source=sdata,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.02},
)

farm = foxes.WindFarm()
foxes.input.farm_layout.add_random(
    farm, n_turbines, min_dist=500, turbine_models=["DTU10MW"], seed=seed, verbosity=0
)

In [ ]:
sdata

In [ ]:
foxes.output.FarmLayoutOutput(farm).get_figure(figsize=(6, 6))
plt.show()

You can run the wind farm calculations by simply creating an algorithm and calling _farm\_calc_:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=1,
)

In [ ]:
farm_results = algo.calc_farm()
farm_results.to_dataframe()

In summary, if you simply run `algo.calc_farm()` (or `algo.calc_points()`or any other function that calls these functions), the _DefaultEngine_ will be created and launched in the background.

As stated in the `calc_farm` printout, the _DefaultEngine_ selected to run the _ProcessEngine_ for this size of problem. The criteria are:

- If `n_states >= sqrt(n_procs) * (500/n_turbines)**1.5`: Run engine `ProcessEngine`,
- Else if `algo.calc_points()` has been called and `n_states*n_points > 10000`: Run engine `ProcessEngine`,
- Else: Run engine `SingleChunkEngine`.

The above selection is based on test runs on a Ubuntu workstation with 64 physical cores and might not be the optimal choice for your system. Be aware of this whenever relying on the default engine for smallish cases - if in doubt, better explicitly specify the engine. This will be explained in the following section.

## Engine selection through a with-block

Engines other than the _DefaultEngine_ are selected by using a Python context manager, i.e., a _with_ block, on an engine objecz. This ensures the proper launch and shutdown of the engine's machinery for parallelization.

The syntax is straight forward. Note that the engine object is not required as a parameter for the algorithm, since it is set as a globally accessible object when entering the _with_ block:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=0,
)

engine = foxes.Engine.new(
    "local_cluster", n_procs=4, chunk_size_states=2000, chunk_size_points=10000
)

with engine:
    farm_results = algo.calc_farm()

    o = foxes.output.FlowPlots2D(algo, farm_results)
    plot_data = o.get_states_data_xy(FV.WS, resolution=30, states_isel=[0])

g = o.gen_states_fig_xy(plot_data, figsize=(6, 6))
next(g)
plt.show()

Notice the _Dashboard_ link which for this particular choice of engine displays the progress and cluster load during the execution. 

## Remarks & recommendations

- Take the time to think about your engine choice, and its parameters. Your choice might matter a lot for the performance of your run.
- In general, all engines accept the parameters `n_procs`, `chunk_size_states`, `chunk_size_points` (the `single` engine ignores them, though).
- If `n_procs` is not set, the maximal number of processes is applied, according to `os.cpu_count()` for Python version < 3.13 and `os.process_cpu_count()` for Python version >= 3.13.
- If `chunk_size_states` is not set, the number of states is divided by `n_procs`. This might be non-optimal for small cases.
- If `chunk_size_points` is not set and there is more than one states chunk, the full number of points is selected such that there is only one point chunk for each state chunk. If there is only one states chunk, the default points chunk size is the number of points divided by `n_procs`.
- In general, for not too small cases, the default `process` engine is a good choice for runs on a linux based laptop or a workstation computer, or within Windows WSL.
- For runs on native Windows, i.e., without WSL, the best engine choices have not been tested. Make sure you try different ones, e.g. `process`, `multiprocess`, `dask`, `numpy`, `ray`, and also vary the parameters.
- The `mpi` engine requires the installation of MPI on the system, for example _OpenMPI_. Don't forget to run this as for example for 12 cores by `mpiexec -n 12 python -m mpi4py.futures run.py`, with `engine = foxes.Engine.new("mpi", n_procs=12, ...)`.
- If you run into memory problems, the best options are to either reduce the number of processes or the chunk sizes.
- The `dask` engine has additional options, accessible through the _dask\_pars_ dictionary parameter, for example the _scheduler_ choice. See API and [dask documentation](https://docs.dask.org/en/stable/scheduling.html) for syntax and more info.
- The `local_cluster` is not always faster than the `process`, `multiprocess` or `dask` engines, but offers a more detailed setup. For example, the memory and the number of threads per worker can be modified, if needed.
- The `numpy` and `single` engines are intended for testing and small cases, and also for sequential runs without large point evaluations, or for smallish runs with wake frame `dyn_wakes`.